# Heuristics vs Mathematical Optimization Using the Binary Paintshop Problem
- In car manufacturing one of the final production steps is painting.
- Multiple cars of different types (A to D) arrive in a given sequence at the paintshop.
 ![sequence](https://github.com/JHasselbringGAMS/Workshop/blob/main/car_sequence.png?raw=1)
- The cars have to be painted with a base coat that is either white or black (here referred to as red or blue).
- The demand for white and black colors for a given car type is also given.

This problem can be simplified to a minimal working example:
- In the sequence of cars arriving at the paintshop each vehicle type arrives exactly twice.
- One car of each vehicle type has to be painted white, the other one has to be painted black.

As changing colors requires time and produces waist, the goal is to minimize the number of color changes with respect to the constraint of coloring one car white and one black for each vehicle type.

This problem can be solved both heuristically or with a mathematical optimization approach.

In [10]:
# Install dependencies
! pip install -q gamspy

In [11]:
! pip install planqk-service-sdk
! git clone https://github.com/GAMS-dev/QUBO.git && cd QUBO/gamspy_qubo && pip install . && pip install -r requirements.txt

fatal: destination path 'QUBO' already exists and is not an empty directory.


In [24]:
import random
random.seed(42)

## Data

In [25]:
types = set(["A", "B"])
n_types = len(types)
sequence = random.choices(list(types), k=5)
print(sequence)
demand_white = {t: random.randint(0, sequence.count(t)) for t in types}
demand_black = {t: sequence.count(t) - demand_white[t] for t in types}
print(f"Demand for black cars: {demand_black}")

['B', 'A', 'A', 'A', 'B']
Demand for black cars: {'A': 3, 'B': 0}


## Mathematical Modeling


We define our constraint

$$\sum_{i: (i,j) \in \mathcal{IJ}} X_i = b_j; \forall \ j \in \mathcal{J}$$

as an Equation. Since it holds for all $j$ the equation is in the domain $j$. Then we can directly use `gp.Sum()` to define the constraint.

The objective
$$\sum_{i \in \mathcal{I} \hspace{0.75mm} | \hspace{0.75mm} i < |I|} (X_i - X_{i+1})^2$$

is defined as an expression.

In [26]:
import gamspy as gp
from gamspy.math import sqr

m = gp.Container()

# create sets
i = gp.Set(m, "i", description="number in sequence")
j = gp.Set(m, "j", description="car type")
IJ = gp.Set(
    m,
    "IJ",
    domain=[i, j],
    records=[(i + 1, sequence[i]) for i in range(len(sequence))],
    domain_forwarding=True,
)

# create variables
X = m.addVariable("X", domain=[i], type="binary", description="color indicator")

black_demand = gp.Parameter(
    m,
    "black_demand",
    domain=[j],
    records=[(type, demand) for type, demand in demand_black.items()],
)

MeetBlackDemand = gp.Equation(m, "MeetBlackDemand", domain=j)
MeetBlackDemand[j] = gp.Sum(IJ[i, j], X[i]) == black_demand[j]

obj = gp.Sum(i.where[gp.Ord(i) < gp.Card(i)], sqr(X[i] - X[i + 1]))

paintshop = gp.Model(
    m,
    name="paintshop",
    problem="MIQCP",
    equations=[MeetBlackDemand],
    # -----------------------------------------------------
    # The model formulation is chosen for the minimzation problem
    # Max here just changes the sign, the solver returns good results
    # for up to 5 cars if its set to MAX, if its set to MIN it fails
    # Not sure what could be the cuase for this problem
    sense=gp.Sense.MAX,
    # -----------------------------------------------------
    objective=obj,
)
paintshop.solve(solver="CPLEX")

classical_string = "".join(["1" if x > 0.5 else "0" for x in X.records["level"]])


In [27]:
import json
import numpy as np
from gamspy_qubo import Qubo
from planqk.service.client import PlanqkServiceClient


paintshop_qubo = Qubo(paintshop, name="paintshop_qubo", penalty=10)
paintshop_qubo.solve()

qubo_matrix = paintshop_qubo.qubo

rows_idx, cols_idx = np.triu_indices(qubo_matrix.shape[0])

values = qubo_matrix[rows_idx, cols_idx]

final_qubo = {}
for r, c, v in zip(rows_idx, cols_idx, values):
    if r == c:
        final_qubo[f"({r},)"] = v
    else:
        final_qubo[f"({r},{c})"] = v

from google.colab import userdata

consumer_key = userdata.get("PlanqkConsumerKey")
consumer_secret = userdata.get("PlanqkConsumerSecret")

free_dcqo_endpoint = "https://gateway.platform.planqk.de/kipu-quantum/kipu-digitized-counterdiabatic-quantum-optimization---dcqo/1.0.0"
premium_dcqo_endpoint = "https://gateway.platform.planqk.de/kipu-quantum/kipu-premium-digitized-counterdiabatic-quantum-optimization---premium-dcqo/1.0.0"
commercial_bias_field_endpoint = "https://gateway.platform.planqk.de/kipu-quantum/kipu-bias-field-digitized-counterdiabatic-quantum-optimization---bfdcqo/1.0.0"

client = PlanqkServiceClient(free_dcqo_endpoint, consumer_key, consumer_secret)

data = {
    "optimization": {
        "coefficients": final_qubo,
        "annealing_time": 0.7,
        "trotter_steps": 2,
        "mode": "CD",
    }
}

params = {"backend": "azure.ionq.simulator", "shots": 1024}

job = client.run(request={"data": data, "params": params})
job_id = job.id

result = client.get_service_execution(job_id)
counts = result.result().dict()["counts"]
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

def calculate_switches(binary_string):
    return sum(1 for i in range(len(binary_string) - 1) if binary_string[i] != binary_string[i+1])


def is_solution_feasible(solution_string):
    if len(sequence) != len(solution_string):
        return False  # The solution must be the same length as the sequence

    black_cars_painted = {car_type: 0 for car_type in set(sequence)}

    for car_type, paint_color in zip(sequence, solution_string):
        if paint_color == "1":
            black_cars_painted[car_type] += 1

    return all(
        black_cars_painted.get(t, 0) == demand_black.get(t, 0)
        for t in set(sequence)
    )


for idx in range(10):
    key = list(counts.keys())[idx]
    count = counts[key]
    objective_value = calculate_switches(key)
    feasible = is_solution_feasible(key)
    print(f"Key: {key}, Count: {count}, Objective Value: {objective_value}, Feasible: {feasible}")

print("Classical Solution:")
key = classical_string
count = counts[key]
objective_value = calculate_switches(key)
feasible = is_solution_feasible(key)
print(f"Key: {key}, Count: {count}, Objective Value: {objective_value}, Feasible: {feasible}")

Key: 01110, Count: 432, Objective Value: 2, Feasible: True
Key: 11111, Count: 192, Objective Value: 0, Feasible: False
Key: 01100, Count: 86, Objective Value: 2, Feasible: False
Key: 11110, Count: 65, Objective Value: 1, Feasible: False
Key: 01111, Count: 43, Objective Value: 1, Feasible: False
Key: 11101, Count: 33, Objective Value: 2, Feasible: False
Key: 01010, Count: 27, Objective Value: 4, Feasible: False
Key: 00010, Count: 19, Objective Value: 2, Feasible: False
Key: 00100, Count: 16, Objective Value: 2, Feasible: False
Key: 01000, Count: 16, Objective Value: 2, Feasible: False
Classical Solution:
Key: 01110, Count: 432, Objective Value: 2, Feasible: True


In [19]:
client = PlanqkServiceClient(premium_dcqo_endpoint, consumer_key, consumer_secret)
job = client.run(request={"data": data, "params": params})
job_id = job.id

result = client.get_service_execution(job_id)
counts = result.result().dict()["counts"]
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
for idx in range(10):
    key = list(counts.keys())[idx]
    count = counts[key]
    objective_value = calculate_switches(key)
    feasible = is_solution_feasible(key)
    print(f"Key: {key}, Count: {count}, Objective Value: {objective_value}, Feasible: {feasible}")

print("Classical Solution:")
key = classical_string
count = counts[key]
objective_value = calculate_switches(key)
feasible = is_solution_feasible(key)
print(f"Key: {key}, Count: {count}, Objective Value: {objective_value}, Feasible: {feasible}")

KeyError: 'counts'

In [28]:
client = PlanqkServiceClient(commercial_bias_field_endpoint, consumer_key, consumer_secret)
data = {
    "bf_dcqo": {
      "coefficients": final_qubo,
      "annealing_time": 0.7,
      "trotter_steps": 2,
      "cd_mode": "CD",
      "bf_mode": "sign",
      "shots": 1024,
      "num_iterations": 4
    }
  }

params = {"backend": "azure.ionq.simulator"}

job = client.run(request={"data": data, "params": params})
job_id = job.id

result = client.get_service_execution(job_id)
counts = result.result().dict()["counts"]
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

for idx in range(10):
    key = list(counts.keys())[idx]
    count = counts[key]
    objective_value = calculate_switches(key)
    feasible = is_solution_feasible(key)
    print(f"Key: {key}, Count: {count}, Objective Value: {objective_value}, Feasible: {feasible}")

print("Classical Solution:")
key = classical_string
count = counts[key]
objective_value = calculate_switches(key)
feasible = is_solution_feasible(key)
print(f"Key: {key}, Count: {count}, Objective Value: {objective_value}, Feasible: {feasible}")

ApiError: headers: {'date': 'Wed, 13 Aug 2025 09:24:46 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'access-control-allow-origin': '*', 'access-control-allow-methods': 'GET', 'x-content-type-options': 'nosniff', 'pragma': 'no-cache', 'access-control-allow-headers': 'authorization,Access-Control-Allow-Origin,Content-Type,apiKey,Authorization', 'x-frame-options': 'DENY', 'activityid': '895ca68f-e3b2-4dc8-9df0-e0457abdd39a', 'access-control-expose-headers': '', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'cache-control': 'no-cache, no-store, max-age=0, must-revalidate', 'vary': 'Access-Control-Request-Headers, Access-Control-Request-Method, Origin', 'expires': '0', 'x-xss-protection': '0'}, status_code: 404, body: {'_embedded': {'status': {'id': 'dfd87f70-9a39-4780-80d9-2a1a9fc62dbf', 'createdAt': '2025-08-13 09:23:39', 'startedAt': '2025-08-13 09:23:44', 'endedAt': '2025-08-13 09:24:19', 'status': 'SUCCEEDED

When I check with the API from the Planqk directly I can see that the Premium DCQO fails (don't know why) and that the BFDCQO claims to succeed, but tryign to access the results fives a 404 not found error.